# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,khandyga,62.6667,135.6000,7.52,96,100,0.57,RU,1725294548
1,1,paredon,16.0333,-93.8667,29.27,68,97,1.82,MX,1725294549
2,2,tazovsky,67.4667,78.7000,6.84,94,100,4.16,RU,1725294550
3,3,puerto ayora,-0.7393,-90.3518,21.97,80,93,0.89,EC,1725294551
4,4,south venice,27.0531,-82.4243,30.01,36,20,7.72,US,1725294552


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 700, frame_height = 500, size = "Humidity", scale = .75, color = "City")

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[city_data_df["Max Temp"] > 21]
city_data_df = city_data_df[city_data_df["Max Temp"] < 27]
city_data_df = city_data_df[city_data_df["Wind Speed"] < 4.5]
city_data_df = city_data_df[city_data_df["Cloudiness"] == 0]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,kasson,44.0300,-92.7507,21.36,51,0,0.45,US,1725294559
56,56,mil'kovo,55.6078,37.7956,24.63,43,0,2.89,RU,1725294615
169,169,deschutes river woods,43.9915,-121.3584,22.01,45,0,2.57,US,1725294751
188,188,zambezi,-13.5432,23.1047,23.83,22,0,2.86,ZM,1725294773
202,202,douglas,41.3000,-96.1670,22.51,52,0,4.12,US,1725294757
226,226,flin flon,54.7682,-101.8650,24.89,32,0,4.36,CA,1725294818
320,320,miles city,46.4083,-105.8406,21.54,52,0,2.57,US,1725294926
399,399,luanda,-8.8368,13.2343,24.00,64,0,4.12,AO,1725295016


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

C:\Users\stlki\AppData\Local\Temp\ipykernel_18108\282354742.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = " "


,City,Country,Lat,Lng,Humidity,Hotel Name
10,kasson,US,44.0300,-92.7507,51,
56,mil'kovo,RU,55.6078,37.7956,43,
169,deschutes river woods,US,43.9915,-121.3584,45,
188,zambezi,ZM,-13.5432,23.1047,22,
202,douglas,US,41.3000,-96.1670,52,
226,flin flon,CA,54.7682,-101.8650,32,
320,miles city,US,46.4083,-105.8406,52,
399,luanda,AO,-8.8368,13.2343,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "hotels",
    "apiKey": geoapify_key,
    "Limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kasson - nearest hotel: No hotel found
mil'kovo - nearest hotel: No hotel found
deschutes river woods - nearest hotel: No hotel found
zambezi - nearest hotel: No hotel found
douglas - nearest hotel: No hotel found
flin flon - nearest hotel: No hotel found
miles city - nearest hotel: No hotel found
luanda - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
10,kasson,US,44.0300,-92.7507,51,No hotel found
56,mil'kovo,RU,55.6078,37.7956,43,No hotel found
169,deschutes river woods,US,43.9915,-121.3584,45,No hotel found
188,zambezi,ZM,-13.5432,23.1047,22,No hotel found
202,douglas,US,41.3000,-96.1670,52,No hotel found
226,flin flon,CA,54.7682,-101.8650,32,No hotel found
320,miles city,US,46.4083,-105.8406,52,No hotel found
399,luanda,AO,-8.8368,13.2343,64,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 700, frame_height = 500, size = "Humidity", scale = .75, color = "City")

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)